In [17]:
import psycopg2
from datetime import datetime
from psycopg2 import sql
import requests
from sentence_transformers import SentenceTransformer
import feedparser
from datetime import datetime, timezone
import re

def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)

        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False
    
def create_conn_source_test():
    conn = None
    try:
        conn = psycopg2.connect(
            host="ep-purple-boat-77462220.ap-southeast-1.aws.neon.tech",
            database="neondb",
            user="gskchaitanya.gadde",
            password="aWO71xgmLjUv"
        )
        print("Connection successful")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def create_conn_destination_test():
    conn = None
    try:
        conn = psycopg2.connect(
            host="db.icrispjgfllulboelvhw.supabase.co",
            database="postgres",
            user="postgres",
            password="p84QS4qighUL0iLV",
            port=5432
        )
        print("Connection to destination successful")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def create_conn_source(select_query):
    conn = None
    try:
        conn = psycopg2.connect(
            host="ep-purple-boat-77462220.ap-southeast-1.aws.neon.tech",
            database="neondb",
            user="gskchaitanya.gadde",
            password="aWO71xgmLjUv"
        )
        cur = conn.cursor()
        cur.execute(select_query)
        rows = cur.fetchall()
        return rows
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()


def select_conn_destination(select_query):
    conn = None
    try:
        conn = psycopg2.connect(
            host="db.icrispjgfllulboelvhw.supabase.co",
            database="postgres",
            user="postgres",
            password="p84QS4qighUL0iLV",
            port=5432
        )
        cur = conn.cursor()
        cur.execute(select_query)
        rows = cur.fetchall()
        return rows
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def create_conn_destination(insert_query, data):
    conn = None

    try:
        conn = psycopg2.connect(
            host="db.icrispjgfllulboelvhw.supabase.co",
            database="postgres",
            user="postgres",
            password="p84QS4qighUL0iLV",
            port=5432
        )
        cur = conn.cursor()
        for item in data:
            #print(item)
            cur.execute(insert_query, item)
        conn.commit()
        cur.close()
        print("Data inserted successfully")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def insert_conn_source(insert_query, data):
    conn = None

    try:
        conn = psycopg2.connect(
            host="ep-purple-boat-77462220.ap-southeast-1.aws.neon.tech",
            database="neondb",
            user="gskchaitanya.gadde",
            password="aWO71xgmLjUv"
        )
        cur = conn.cursor()
        for item in data:
            #print(item)
            cur.execute(insert_query, item)
        conn.commit()
        cur.close()
        print("Data inserted successfully")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def update_conn_source(update_query):
    conn = None
    try:
        conn = psycopg2.connect(
            host="ep-purple-boat-77462220.ap-southeast-1.aws.neon.tech",
            database="neondb",
            user="gskchaitanya.gadde",
            password="aWO71xgmLjUv"
        )
        cur = conn.cursor()
        cur.execute(update_query)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def update_conn_destination(update_query):
    conn = None

    try:
        conn = psycopg2.connect(
            host="db.icrispjgfllulboelvhw.supabase.co",
            database="postgres",
            user="postgres",
            password="p84QS4qighUL0iLV",
            port=5432
        )
        cur = conn.cursor()
        cur.execute(update_query)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

#create_conn_source_test()
#create_conn_destination_test()


In [29]:


articles_from_rss = """
SELECT 
    rss1_link,
    rss1_link_name,
    outlet,
    id
FROM 
    synopse_articles.t_v1_rss1_feed_links
WHERE 
    rss1_link_type = 11 AND 
    outlet = 'aljazeera';
"""
articles_l2_details_output = create_conn_source(articles_from_rss)
for l1 in articles_l2_details_output:
    NewsFeed = feedparser.parse(l1[0])
    v1 = []
    for entry in NewsFeed.entries:
        post_link = entry.link
        title = entry.title
        summary = ''
        is_in_detail = 0
        if 'summary' in entry:
            summary_nofil = entry.summary
            summary = re.sub('<[^<]+?>', '', summary_nofil)
            is_in_detail = 1
        image_url = ""
        if 'media_content' in entry:
            try:
                if entry['media_content']:
                    image_url = entry['media_content'][0]['url']
            except:
                image_url = ''
        if 'links' in entry:
            for link in entry.links:
                if link.type == "image/jpeg":
                    image_url= link.href
                    break
        published = datetime.now(timezone.utc).isoformat()
        if 'published' in entry:
            published = entry.published
        datevalidation = is_valid_timezone_format(published)
        if datevalidation[0]:
            hasura_timestamp = datevalidation[1]
        if check_date_format(published):
            hasura_timestamp = published
        else:
            hasura_timestamp = datetime.now().astimezone(timezone.utc).isoformat()
        if "author" in entry:
            author = entry.author
        else:
            author = "na"
        tags = []
        tags.append(l1[1])
        tags.append(l1[2])
        if 'tags' in entry:
            for tag in entry.tags:
                tags.append(tag.term)
        v2 = []
        v2.append(l1[3])
        v2.append(post_link)
        v2.append(title)
        v2.append(summary)
        v2.append(author)
        v2.append(image_url)
        v2.append(hasura_timestamp)
        v2.append(is_in_detail)
        v1.append(tuple(v2))
    
    v1_insert_query ="INSERT INTO synopse_articles.t_v1_rss1_articles (rss1_feed_id, post_link, title, summary, author, image_link, post_published, is_in_detail) VALUES (%s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING;"
    
    insert_conn_source(v1_insert_query, v1)
    print(insert_conn_source)

duplicate key value violates unique constraint "t_v1_rss1_articals_post_link_key"
DETAIL:  Key (post_link)=(https://www.aljazeera.com/opinions/2024/1/4/for-feminists-silence-on-gaza-is-no-longer-an-option?traffic_source=rss) already exists.

<function insert_conn_source at 0x0000021DAF0693A0>


In [28]:
def a1(outlet):
    articles_from_rss = """
    SELECT 
        rss1_link,
        rss1_link_name,
        outlet,
        id
    FROM 
        synopse_articles.t_v1_rss1_feed_links
    WHERE 
        rss1_link_type = 11 AND 
        outlet = '""" + outlet + """';
    """
    print(articles_from_rss)

q1 = """select outlet from synopse_articles.t_v1_outlets;"""

q1_out = create_conn_source(q1)
outlets = [item[0] for item in q1_out]
print(outlets)
for outlet in outlets:
    a1(outlet)

['timesofindia', 'thehindu', 'cnn', 'foxnews', 'cbsnews', 'espn', 'forbes', 'abcnews', 'huffpost', 'politico', 'independent', 'techcrunch', 'the verge', 'nytimes', 'reuters', 'vox', 'nbc', 'wsj', 'thewashingtonpost', 'aljazeera', 'wired', 'marketwatch', 'newyorkpost', 'appleinsider', 'thewire', 'wion', 'theindianexpress', 'hindustantimes']

    SELECT 
        rss1_link,
        rss1_link_name,
        outlet,
        id
    FROM 
        synopse_articles.t_v1_rss1_feed_links
    WHERE 
        rss1_link_type = 11 AND 
        outlet = 'timesofindia';
    

    SELECT 
        rss1_link,
        rss1_link_name,
        outlet,
        id
    FROM 
        synopse_articles.t_v1_rss1_feed_links
    WHERE 
        rss1_link_type = 11 AND 
        outlet = 'thehindu';
    

    SELECT 
        rss1_link,
        rss1_link_name,
        outlet,
        id
    FROM 
        synopse_articles.t_v1_rss1_feed_links
    WHERE 
        rss1_link_type = 11 AND 
        outlet = 'cnn';
    

    SEL

In [ ]:
import os
import re
import time


def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)
        
        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False

    


In [35]:
def update_article_details1(offset):
    q1 = """select a.id, a.post_link 
        from synopse_articles.t_v1_rss1_articles a 
        where a.is_in_detail = 0 
        order by a.post_published
        LIMIT 1 OFFSET """ + str(offset) + """;"""
    while True:
        q1_out = create_conn_source(q1)
        if len(q1_out) == 0:
            return 0
        id = ""
        post_link = ""
        for item in q1_out:
            id = item[0]
            post_link = item[1]
        break

update_article_details1(0)